In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import re
import getpass
import pandas as pd
import time

In [ ]:
# Setup Selenium WebDriver
driver_path = r"C:\Users\puvia\.cache\selenium\chromedriver\win64\130.0.6723.116\chromedriver.exe"  # Replace with your WebDriver path

In [27]:

# Function to login to Amazon
def amazon_login(email, password):
    driver.get("https://www.amazon.in/ap/signin?openid.pape.max_auth_age=900&openid.return_to=https%3A%2F%2Fwww.amazon.in%2Fgp%2Fyourstore%2Fhome%3Fpath%3D%252Fgp%252Fyourstore%252Fhome%26signIn%3D1%26useRedirectOnSuccess%3D1%26action%3Dsign-out%26ref_%3Dnav_AccountFlyout_signout&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ap_email')))
    
    driver.find_element(By.ID, 'ap_email').send_keys(email)
    driver.find_element(By.ID, 'continue').click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ap_password')))
    
    driver.find_element(By.ID, 'ap_password').send_keys(password)
    driver.find_element(By.ID, 'signInSubmit').click()


In [35]:

# Function to navigate to product reviews
def navigate_to_reviews(product_url):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10)
    driver.get(product_url)
    WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-hook='see-all-reviews-link-foot']"))
).click()




In [49]:
# Function to scrape reviews
def scrape_reviews():
    reviews = []
    while True:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_blocks = soup.find_all('div', {'data-hook': 'review'})
        
        for review in review_blocks:
            try:
                username = review.find('span', {'class': 'a-profile-name'}).get_text(strip=True)
                review_text = review.find('span', {'data-hook': 'review-body'}).get_text(strip=True)
                
                # Extract review date and isolate the date part
                full_review_date = review.find('span', {'data-hook': 'review-date'}).get_text(strip=True)
                # Use a regular expression to capture only the date
                date_match = re.search(r'\b\d{1,2} [A-Za-z]+ \d{4}\b', full_review_date)
                review_date = date_match.group(0) if date_match else "Unknown Date"
                
                #extract rating and isolate it
                full_rating_text = review.find('span', {'class': 'a-icon-alt'}).get_text(strip=True)
                # Use a regular expression to capture only the rating
                rate_match = re.search(r'(\d+(\.\d+)?)', full_rating_text)
                rating = rate_match.group(0) if rate_match else "N/A"
                
                reviews.append({'Username': username, 'Date': review_date, 'Rating': rating, 'Review': review_text})
            except AttributeError:
                continue
        
        # Navigate to the next page of reviews if available
        try:
            next_button = driver.find_element(By.CLASS_NAME, 'a-last')
            if 'a-disabled' in next_button.get_attribute('class'):
                break
            next_button.click()
            time.sleep(3)
        except Exception:
            break

    return reviews



**-----------------------------------------------------------------------------------AMAZON SCRAPPER-----------------------------------------------------------------------------------**

In [56]:
#start the driver
driver = webdriver.Chrome(service=Service(driver_path))
driver.maximize_window()

In [57]:
# Main program

email = input("Enter your username: ")  
password = getpass.getpass("Enter your password: ") 
product_url = input("Enter the Amazon product URL: ") 

try:
    amazon_login(email, password)
    navigate_to_reviews(product_url)
    reviews_data = scrape_reviews()
    
    # Save data to CSV
    df = pd.DataFrame(reviews_data)
    df.to_csv('amazon_reviews.csv', index=False)
    print("Reviews saved to 'amazon_reviews.csv'")
finally:
    driver.quit()

Enter your username:  6383341344
Enter your password:  ········
Enter the Amazon product URL:  https://www.amazon.in/OnePlus-Super-Silver-128GB-Storage/dp/B0D5YCYS1G/ref=sr_1_4?crid=2WWJGR9UOWY8V&dib=eyJ2IjoiMSJ9.3bzUr-Fo8mAuZzivaSli3LZE7_uuaro9yNEOcAV4PohublICuVT2P73dB3Ppyknv0WoKm8D3-sfHNq81ZJL5rFh6jhqW2ZTLwdfIR39k_dKdU_0-SUTivTF-kPTCombdUs1fh6_mPR5N1T6IAQdKm8yIpVGebK1OTc6NusyHpIN4soRdlnDls0kX94QNEpJAD0rZFz6lT0FublQt6MejgSYB1HdNej-0553Y5lvgBcg.jHrUHa0MxJH5QqM5iW22g-Bm_NsSPlCS_YACkw9DVTE&dib_tag=se&keywords=asus%2Brog%2Bphone&nsdOptOutParam=true&qid=1732723839&sprefix=asus%2Brog%2Bphone%2Caps%2C281&sr=8-4&th=1


Reviews saved to 'amazon_reviews.csv'
